![example](images/director_shot.jpeg)

# Navigating Aviation Risks for Business Expansion

**Authors:** Kigen Tuwei
***

## Overview

In this project, we analyze aviation accident data to help a company decide which aircraft types are the safest for their new aviation business. The dataset, sourced from the National Transportation Safety Board, contains civil aviation accident records from 1962 to 2023.  

Using **data cleaning, imputation, exploratory analysis, and visualization**, we identify key risk factors associated with different aircraft models, flight purposes, weather conditions, and accident phases. Our findings provide **three actionable business recommendations** to guide the company in selecting low-risk aircraft.  


## Data Understanding

In this project, we use aviation accident data [AviationData.csv](./AviationData.csv) from the **National Transportation Safety Board (NTSB)**, which contains records of civil aviation accidents from **1962 to 2023**. The dataset provides insights into **aircraft types, accident causes, weather conditions, and flight phases** all crucial for assessing aviation risks. We also have state code data [USState_Codes.csv](./USState_Codes.csv) for location mapping 


In [33]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [34]:
# Load datasets
aviation_df = pd.read_csv("AviationData.csv", encoding = "latin1", low_memory = False)
state_codes_df = pd.read_csv("USState_Codes.csv", encoding = "latin1", low_memory = False)

# Display the first 5 rows
aviation_df.head()

# We notice quite a few null values(NaN)

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


### Check the Structure of the DataFrame

1. Provides number of rows and columns.
2. Helps identify data types and potential inconsistencies.
3. Reveals missing values.

**using 'aviation_df.info()'**

In [35]:
aviation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

### Summary Statistics
 
1. To understand the distribution of numerical variables.
2. Shows the minimum and maximum values.
3. Highlights potential outliers.

**using 'aviation_df.describe()'**

In [36]:
aviation_df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


### Check for Missing Values

1. Helps determine whether to drop, fill, or impute missing data based on relevance.

**using 'aviation_df.isnull().sum()'**

In [37]:
aviation_df.isnull().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38640
Airport.Name              36099
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1317
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7077
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6381
Publication.Date          13771
dtype: i

## Data Preparation

Below are some of the processes I'll be using to clean the data to ensure a higher degree of accuracy and to provide better insights:

1. **Handling Missing Values**: I'll be dropping columns with excessive nulls and replacing them using median (for numeric values) and mode (for categoric values).    
2. **Feature Engineering**: New variables such as "Injury Severity Score" and "Risk Level" will be created to better assess accident severity.  
3. **Merging Datasets**: The aviation dataset will be merged with the U.S. state codes dataset to provide location-based insights.  


### Handling Missing Values

1. I'll drop columns with too many missing values (> 40% missing)
2. I'll also fill missing numerical values with the median and missing categorical values with the mode.

In [38]:
# Dropping columns with too many missing values (more than 40% missing)
missing_threshold = 0.4 * len(aviation_df)  # 40% of total rows
aviation_df = aviation_df.dropna(thresh=missing_threshold, axis=1)


# Filling missing values in numerical columns with the median
num_cols = [col for col in aviation_df.columns if aviation_df[col].dtype in ['int64', 'float64']] # finding the numeric columns
for column in num_cols:
    aviation_df[column].fillna(aviation_df[column].median(), inplace=True) # filling missing values with median

# Filling missing values in categorical columns with the mode
cat_cols = [col for col in aviation_df.columns if aviation_df[col].dtype == 'object'] # finding categorical columns
for column in cat_cols:
    aviation_df[column].fillna(aviation_df[column].mode()[0], inplace=True) # filling the missing values with the mode in the first index [0]


### Feature Engineering

This involves creating two new columns;

1. 'Injury_Severity_Score' to measure accident severity
2. 'Risk_level' column based on severity

In [39]:
# Create an "Injury Severity Score" to measure accident severity
# By combining their weighted values
aviation_df["Injury_Severity_Score"] = (
    aviation_df["Total.Fatal.Injuries"].fillna(0) * 3 +  # Fatal injuries are the most severe with a weight of 3
    aviation_df["Total.Serious.Injuries"].fillna(0) * 2 +  # Serious injuries are given medium weight of 2
    aviation_df["Total.Minor.Injuries"].fillna(0) * 1  # Minor injuries are the least severe with a weight of 1
)

# Create a "Risk Level" column based on severity; High Risk, Medium Risk and Low Risk
def categorize_risk(score):
    if score >= 10:
        return "High Risk"
    elif score >= 5:
        return "Medium Risk"
    else:
        return "Low Risk"

aviation_df["Risk_Level"] = aviation_df["Injury_Severity_Score"].apply(categorize_risk)

aviation_df[["Injury_Severity_Score", "Risk_Level"]].head(15) # to display accuracy of changes

,Injury_Severity_Score,Risk_Level
0,6.0,Medium Risk
1,12.0,High Risk
2,9.0,Medium Risk
3,6.0,Medium Risk
4,7.0,Medium Risk
5,1.0,Low Risk
6,12.0,High Risk
7,0.0,Low Risk
8,0.0,Low Risk
9,3.0,Low Risk


### Merging Datasets

1. To improve location-based analysis, we will merge `aviation_df` with `state_codes_df` on the state column.

In [43]:
# Merging Datasets
# Rename "Location" column to "State" for consistency
aviation_df.rename(columns={"Location": "State"}, inplace=True)

# Merge aviation dataset with state codes dataset
aviation_df = aviation_df.merge(state_codes_df, how="left", on="State")

# Display the first few rows to check changes
aviation_df.head(13)

,Event.Id,Investigation.Type,Accident.Number,Event.Date,State,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,...,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Injury_Severity_Score,Risk_Level,Abbreviation
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NONE,Private,Fatal(2),Destroyed,...,0.0,0.0,0.0,UNK,Cruise,Probable Cause,25-09-2020,6.0,Medium Risk,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NONE,Private,Fatal(4),Destroyed,...,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996,12.0,High Risk,NaN
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,NONE,Private,Fatal(3),Destroyed,...,0.0,0.0,1.0,IMC,Cruise,Probable Cause,26-02-2007,9.0,Medium Risk,NaN
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NONE,Private,Fatal(2),Destroyed,...,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000,6.0,Medium Risk,NaN
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NONE,Private,Fatal(1),Destroyed,...,2.0,0.0,0.0,VMC,Approach,Probable Cause,16-04-1980,7.0,Medium Risk,NaN
5,20170710X52551,Accident,NYC79AA106,1979-09-17,"BOSTON, MA",United States,NONE,Private,Non-Fatal,Substantial,...,0.0,1.0,44.0,VMC,Climb,Probable Cause,19-09-2017,1.0,Low Risk,NaN
6,20001218X45446,Accident,CHI81LA106,1981-08-01,"COTTON, MN",United States,NONE,Private,Fatal(4),Destroyed,...,0.0,0.0,0.0,IMC,Unknown,Probable Cause,06-11-2001,12.0,High Risk,NaN
7,20020909X01562,Accident,SEA82DA022,1982-01-01,"PULLMAN, WA",United States,NONE,BLACKBURN AG STRIP,Non-Fatal,Substantial,...,0.0,0.0,2.0,VMC,Takeoff,Probable Cause,01-01-1982,0.0,Low Risk,NaN
8,20020909X01561,Accident,NYC82DA015,1982-01-01,"EAST HANOVER, NJ",United States,N58,HANOVER,Non-Fatal,Substantial,...,0.0,0.0,2.0,IMC,Landing,Probable Cause,01-01-1982,0.0,Low Risk,NaN
9,20020909X01560,Accident,MIA82DA029,1982-01-01,"JACKSONVILLE, FL",United States,JAX,JACKSONVILLE INTL,Non-Fatal,Substantial,...,0.0,3.0,0.0,IMC,Cruise,Probable Cause,01-01-1982,3.0,Low Risk,NaN


## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***